PREPROCESSING

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import datetime
from dateutil.relativedelta import relativedelta

In [30]:
raw_data = pd.read_csv('../csv/raw/caspecoTrainingData.csv', header=0)
holiday_data = pd.read_csv('../csv/raw/swedish_holidays.csv', header=0)
raw_data = raw_data.sort_values('Date').reset_index(drop=True)
raw_data.head()
holiday_data.head()

,Date,Day
0,2020-01-01,New Year Day
1,2020-01-06,Epiphany
2,2020-04-10,Good Friday
3,2020-04-12,Easter Day
4,2020-04-13,Easter Monday


In [34]:
'Utility'

def df_convert_dates(df):
    df["Date"] = pd.to_datetime(df["Date"])
    df["Date"] = df["Date"].dt.date
    return df

In [32]:
'Base processing'

def missing_dates(df):
    date_counts = df["Date"].value_counts()[df["Date"].sort_values()].to_dict()
    t = np.datetime_as_string(np.arange(datetime.datetime(2020,1,1), datetime.datetime(2023,1,4), datetime.timedelta(days=1)),unit="D")
    missing_dates = []
    for date in t:
        if date not in date_counts :
            missing_dates.append(date)
        if date_counts[date] < 3:
            missing_dates.append(date)
    return missing_dates

def add_dates(lst, df):
    
    comp = np.sort(df['Company'].unique())

    for date in lst:

        new_df = df[df['Date'] == date]

        for c in comp:

            if c not in new_df['Company'].values:  
                new_row = pd.DataFrame({"Date": [date], "Company": [c],"Sales": [-1]})
                df = pd.concat([df, new_row], ignore_index=True)
    
    df = df.sort_values('Date').reset_index(drop=True)
    return df

def add_holidays(df, df_holiday):

    if 'is_holiday' in df.columns:
        return df
    
    df['is_holiday'] = 0

    for date in df_holiday['Date']:
        df.loc[df['Date'] == date,'is_holiday'] = 1

    return df

def add_weekdays(df):

    if 'weekday' in df.columns:
        return df
    
    df['weekday'] = 0

    for i, date in enumerate(df['Date']):

        df.at[i, 'weekday'] = datetime.datetime.strptime(date, "%Y-%m-%d").weekday()

    return df

def create_train_base_csv():

    missing = missing_dates(raw_data)
    df_dates = add_dates(missing, raw_data)
    df_holiday = add_holidays(df_dates, holiday_data)
    df_days = add_weekdays(df_holiday)
    df_days.to_csv('../csv/processed/train_base.csv', index=False)

create_train_base_csv()

In [54]:
'Average sales imputation (3 months)'
base_data = pd.read_csv("../csv/processed/train_base.csv")

'Currently calcs average for past, current and next month'
def calc_average_sales(df :pd.DataFrame, row):
    months = 3
    df = df[(df["Sales"] != -1) & (df["weekday"] == row["weekday"]) & (df["Company"] == row["Company"])]

    date = row["Date"]
    
    start_date = (date - relativedelta(months=months-2)).replace(day=1)
    end_date = (date + relativedelta(months=months-2)) + relativedelta(day=31)

    if date.year == 2020 and date.month == 1:
        start_date = date
        end_date = (date + relativedelta(months=months)) + relativedelta(day=31)
    elif (date.year == 2023 and date.month == 1) or (date.year == 2022 and date.month == 12):
        start_date = (date - relativedelta(months=months)).replace(day=1)
        end_date = date
        
    rows_in_range = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

    return rows_in_range["Sales"].sum() / rows_in_range.shape[0]

def add_average_sales(df):
    df = df_convert_dates(df)
    missing_sales_rows = df[df["Sales"] == -1]
    for i, row in missing_sales_rows.iterrows():
        avg_sales = calc_average_sales(df, row)
        df.at[i, "Sales"] = avg_sales
    return df

df_avg_sales = add_average_sales(base_data)
df_avg_sales.to_csv('../csv/processed/train_avg_sales.csv', index=False)

